In [1]:
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 
sys.path.insert(0, os.path.join(parentdir, 'utils') )

In [2]:
!pip install psutil

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install colorama

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import config
import csv
from tqdm import tqdm
import itertools as it, glob
import numpy as np

paths from config ['/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/utils', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/data_utils/data_loaders', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/models', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/trainers', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/data_utils', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/utils', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments', '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/jupyter', '/software/jupyterlab/opt/anaconda3/envs/tensorflow/lib/python38.zip', '/software/jupyterlab/opt/anaconda3/envs/tensorflow/lib/python3.8', '/software/jupyterlab/opt/anaconda3/envs/tensorflow/lib/python3.8/lib-dynload', '', '/home/sc.uni-leipzig.de/mi186veva/.local/lib/python3.8/site-packages', '/software/jupyterlab/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages']
FILES_TO_COPY ['/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/utils.py', '/home/sc.uni-leipzig.de/mi186ve

In [5]:
def glob_multiple_file_types(path, *patterns):
    return list(it.chain.from_iterable(glob.iglob(os.path.join(path, pattern)) for pattern in patterns))


def round_to_the_nearest_even_int(number, nearest_int=config.WRITE_CHECKPOINT_EVERY_Xth_STEP):
    return int(int(number / nearest_int) * nearest_int)

In [13]:
def get_history(model_path):
    history_paths = glob_multiple_file_types(model_path, '.*.npy', '*.npy')
    #print(history_paths)
    if len(history_paths) == 0:
        print('Error! No history files were found!')
        #raise ValueError('Error! No history files were found!')
        return {}, model_path
    if len(history_paths) > 1:
        print(f'Error! Too many .npy files were found in {model_path}!')
        return {}, model_path
        #raise ValueError(f'Error! Too many .npy files were found in {model_path}!')

    history_path = history_paths[0]
    history = np.load(history_path, allow_pickle=True)
    #print(history)
    if len(history.shape) == 0:
        history = history.item()
    #print(history)
    return history, history_path

def get_best_checkpoint_from_valid(results_file, nearest_int=config.WRITE_CHECKPOINT_EVERY_Xth_STEP):
    model_paths = []
    best_checkpoints = []

    with open(results_file, newline='') as csvfile:
        report_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in tqdm(report_reader):
            model_path = row[5]
            if 'LOCAL' in config.MODE:
                model_path = row[5].split('hsi-experiments')[-1][1:]

            model_paths.append(model_path)

            history, history_path = get_history(model_path)
            if not bool(history):
                print(f'Attention!! {history_path} is empty!!!')
                continue
            #best_checkpoint = np.argmin(history[config.HISTORY_ARGMIN])
            best_checkpoint = np.argmax(history['val_accuracy'])
            
            best_checkpoints.append(best_checkpoint)

    if len(best_checkpoints) == 0:
        return -1, best_checkpoints, model_paths

    best_checkpoint = round_to_the_nearest_even_int(np.median(best_checkpoints), nearest_int = nearest_int)

    return best_checkpoint, best_checkpoints, model_paths

In [14]:
best_checkpoint, best_checkpoints, model_paths = get_best_checkpoint_from_valid('/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/logs/CV_3d_inception/CV_3d_inception_stats_07.12.2021-00_01_56.csv')


56it [00:00, 690.26it/s]


In [12]:
best_checkpoints, best_checkpoint

([39,
  39,
  39,
  39,
  37,
  37,
  37,
  37,
  35,
  35,
  35,
  35,
  26,
  26,
  26,
  26,
  38,
  38,
  38,
  38,
  39,
  39,
  39,
  39,
  36,
  36,
  36,
  36,
  36,
  36,
  36,
  36,
  37,
  37,
  37,
  37,
  39,
  39,
  39,
  39,
  38,
  38,
  38,
  38,
  36,
  36,
  36,
  36,
  29,
  29,
  29,
  29,
  24,
  24,
  24,
  24],
 36)

In [15]:
best_checkpoints, best_checkpoint

([39,
  39,
  39,
  39,
  37,
  37,
  37,
  37,
  35,
  35,
  35,
  35,
  25,
  25,
  25,
  25,
  38,
  38,
  38,
  38,
  39,
  39,
  39,
  39,
  35,
  35,
  35,
  35,
  36,
  36,
  36,
  36,
  37,
  37,
  37,
  37,
  38,
  38,
  38,
  38,
  38,
  38,
  38,
  38,
  36,
  36,
  36,
  36,
  29,
  29,
  29,
  29,
  24,
  24,
  24,
  24],
 36)

In [17]:
gt_compiled = []
predictions_compiled = []
prefix = '/home/sc.uni-leipzig.de/mi186veva/hsi-experiments/test/'

folder = 'CV_3d_inception'

for pat_number, checkpoint in tqdm(enumerate(best_checkpoints)):
    checkpoint = round_to_the_nearest_even_int(checkpoint, nearest_int = 2)
    
    pred = np.load(os.path.join(prefix, folder, f'cp-{checkpoint:04d}', 'predictions_by_patient.npy'), allow_pickle=True)
    gt = np.load(os.path.join(prefix, folder, f'cp-{checkpoint:04d}', 'gt_by_patient.npy'), allow_pickle=True)
    
    gt_compiled.append(gt[pat_number])
    predictions_compiled.append(pred[pat_number])
    
print(len(gt_compiled), len(predictions_compiled))

save_path = os.path.join(prefix, folder, 'compiled')
if not os.path.exists(save_path):
    os.mkdir(save_path)
    
np.save(os.path.join(save_path, 'predictions_by_patient'), predictions_compiled)
np.save(os.path.join(save_path, 'gt_by_patient'), gt_compiled)

    

56it [11:08, 11.94s/it]


56 56


In [20]:
round_to_the_nearest_even_int(5)

4

In [21]:
int(np.round(5 / 2) * 2)

4

In [26]:
int(55 / 50) * 50

50

In [21]:
import itertools
import pandas as pd
a = [["l2_norm", "svn"],["with_sample_weights", "without_sample_weights"],["exclude_1", "exclude_4"], ["every_third", "all"]]
g = list(itertools.product(*a))
g_ = pd.DataFrame(g, columns=["norm", "sample_weights", "exclude", "e3_all"])

In [22]:
from IPython.display import display
display(g_)

,norm,sample_weights,exclude,e3_all
0,l2_norm,with_sample_weights,exclude_1,every_third
1,l2_norm,with_sample_weights,exclude_1,all
2,l2_norm,with_sample_weights,exclude_4,every_third
3,l2_norm,with_sample_weights,exclude_4,all
4,l2_norm,without_sample_weights,exclude_1,every_third
5,l2_norm,without_sample_weights,exclude_1,all
6,l2_norm,without_sample_weights,exclude_4,every_third
7,l2_norm,without_sample_weights,exclude_4,all
8,svn,with_sample_weights,exclude_1,every_third
9,svn,with_sample_weights,exclude_1,all


In [ ]:
import csv
